In [1]:
import os
import subprocess
import json
import hashlib
from firebase_admin import credentials, initialize_app, storage

In [2]:
root_dir = "/home/jesuraj/Desktop/bengali_utt_test/audios/"
dest_dir = "/home/jesuraj/Desktop/bengali_utt_test/mp3_audios/"

In [3]:
all_wav_files = []
for path, subdirs, files in os.walk(root_dir):
    for name in files:
        all_wav_files.append(os.path.join(path, name))

all_wav_files = [x for x in all_wav_files if x[-3:] == "wav" ]

for wav_file in all_wav_files:
    subprocess.run("sox " + wav_file + " -t mp3 " + dest_dir + wav_file.split("/")[-1][0:-4] + ".mp3  norm -0.1", shell=True)

In [4]:
cred = credentials.Certificate("/home/jesuraj/Desktop/bengali_utt_test/bengali-utt-test-859817a0a40c.json")
#initialize_app(cred, {'storageBucket': 'hindi-speaker-similarity-check.appspot.com', 'name': 'hindi-speaker-similarity-check'}, name='hindi-speaker-similarity-check')
initialize_app(cred, {'storageBucket': 'bengali-utt-test.appspot.com'})

all_files = []
for path, subdirs, files in os.walk(dest_dir):
    for name in files:
        all_files.append(os.path.join(path, name))

for fname in all_files: 
    bucket = storage.bucket()
    blob = bucket.blob("mp3_audios/" + fname.split("/")[-1])
    blob.upload_from_filename(fname)

In [5]:
with open("/home/jesuraj/Desktop/bengali_utt_test/text") as fp:
    text_maps = fp.readlines()

with open("/home/jesuraj/Desktop/bengali_utt_test/wav.scp") as fp:
    wav_maps = fp.readlines()



In [6]:
text_maps[0].split(" ",1)
wav_maps[0].split(" ", 1)

['16777683_641904_281474982077348',
 '/data/Database/RESPIN/Bengali/bn-100/281474982077348.wav\n']

In [16]:
filemap_dict = {}

for text_map in text_maps:
    filemap_dict[text_map.split(" ", 1)[0]] = [text_map.split(" ", 1)[1].strip()]

for wav_map in wav_maps:
    filemap_dict[wav_map.split(" ", 1)[0]].append("mp3_audios/" + wav_map.split(" ", 1)[1].strip().split("/")[-1][:-4] + ".mp3")


In [ ]:
filemap_dict

In [22]:
json_dict = {"mp3path": {}, "textpath": {}}

In [23]:
for index in filemap_dict.keys():
    json_dict["mp3path"][index] = filemap_dict[index][1]
    json_dict["textpath"][index] = filemap_dict[index][0]

In [25]:
with open("/home/jesuraj/Desktop/bengali_utt_test/bengali-utt-test/src/new_utt_path.json", "w") as fp:
    json.dump(json_dict, fp, ensure_ascii=False, indent=True)

In [ ]:
counts = {"count": {}}
for i in range(0,len(filemap_dict),100):
    counts["count"][str(i)] = 0

with open("/home/jesuraj/Desktop/bengali_utt_test/bengali-utt-test/count.json", "w") as fp:
    json.dump(json_dict, fp, ensure_ascii=False, indent=True)